In [720]:
import urllib.request
import urllib.parse
import urllib.error
from bs4 import BeautifulSoup
import ssl
import json
import ast
import json
import os
from urllib.request import Request, urlopen
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
import pandas as pd
from selenium.webdriver.chrome.options import Options
import requests
from langdetect import detect
import re
import datetime
from datetime import date
from datetime import datetime, timedelta

# Create equation research

In [564]:
compagniesAeriennes=["American Airlines","Air Canada","Air France","Air Algerie","Air India","Aerolineas Argentinas","Royal Air Maroc","Finnair" ,"Alitalia "," Nouvelair","Air China","Cathay Pacific","Delta Airlines","Aer Lingus","Emirates","Ethiopian Airlines","Icelandair","Hawaiian Airlines","Iberia","Meridiana","Japan Airlines","KLM","Air Malta","Lan Airlines","Luxair","LIAT","LOT Polish Airlines","Air Madagascar","Air Mauritius","Austrian Airlines","Qatar Airways","South African Airways","SAS Scandinavian Airlines","Brussels Airlines","Singapore Airlines","Corsair","Aeroflot","Thai Airways","Turkish Airlines","TAP Portugal","Air Transat","Tunisair","Air Caraibes","United Airlines","Air Austral","Air Europa","Easyjet","Vietnam Airlines","Virgin Atlantic","Air Corsica","Condor","Flybe","Aegean Airlines","Air Tahiti Nui","Aigle Azur","HOP!","Jet Airways","Etihad Airways","Etihad Airways","Oman Air","XL Airways","Ryanair LTD","Vueling ","Norwegian","Transavia France","Germanwings","TUI Fly Belgium","Air Arabia","WOW air","Wizz Air","Air Asia","Volotea","southwest airlines"]
compagniesAeriennes = [compagnies.replace(' ','+') for compagnies in compagniesAeriennes]

modelesBoeing=["Boeing 717","Boeing 727","Boeing 737-200","Boeing 737-300","Boeing 737-400","Boeing 737-500","Boeing 737-600","Boeing 737-700","Boeing 737-700ER","Boeing 737-800","Boeing 737-900","Boeing 737-900ER","Boeing 737 MAX 7","Boeing 737 MAX 8","Boeing 737 MAX 9","Boeing 737 MAX 10","Boeing 747-200","Boeing 747-400","Boeing 757-200","Boeing 757-300","Boeing 767-200","Boeing 767-300","Boeing 767-300ER","Boeing 767-400ER","Boeing 777 Triple Seven","Boeing 787 DreamLiner"]
modelesBoeing = [modele.replace(' ','+') for modele in modelesBoeing]

modelesAirbus=["A300","A300-600ST","A318","A319","A320-100","A320-200","A320neo","A321-100","A321-200","A330-200","A330-300","A330-200F","A330-500","A340-200","A340-300","A340-500","A340-600","A350-900","A350-1000","A380-800","A220-300"]
motsCles=["trip","fly","plane","airplane","flight"]


equations=[]
for comp in compagniesAeriennes:
    for mod in modelesAirbus:
        equations.append(comp+"+"+mod)
    for mod in modelesBoeing:
        equations.append(comp+"+"+mod)
print(len(equations))

3431


# fonction chipottage

In [786]:
def modifierDate(chaine):
    expression=re.search("[0-9]*\s[a-zA-Z]*$" ,chaine)
    return(expression.group(0))

def Simplification(chaine):
    chaine = chaine.replace(',','.')
    if '.' in chaine :
        expression=(re.search("\d+\.\d+" ,chaine)).group(0)
    else : 
        expression=(re.search("\d+" ,chaine)).group(0)
    facteur = 1
    if 'k' in chaine : 
        facteur = 1000
    if 'M' in chaine : 
        facteur = 1000000
    return(int(float(expression) * facteur))


def fctdate1(chaine):
    chaine=(chaine).replace('il y a ','')
    if 'hier' in chaine : 
        nb_jour = 1
    else : 
        nb_jour = int((re.search("\d+" ,chaine)).group(0))
    if 'mois' in chaine: 
        nb_jour = nb_jour * 30
    if 'ans' in chaine : 
        nb_jour = nb_jour * 365
    if 'semaine' in chaine :
        nb_jour = nb_jour * 7
        
    now = date.today()
    return str(now - timedelta(days = (nb_jour)))

# Création de la liste URL

In [439]:
def listeUR (EquationRecherche) :
    racineURL = "https://www.youtube.com/results?search_query="
    #EquationRecherche = "airbus+A380"

    r = requests.get(racineURL + EquationRecherche)
    page = r.text
    soup = bs(page,'html.parser')

    videos = soup.findAll('a',attrs={'class':'yt-uix-tile-link'})
    listVideos=[]
    for v in videos:
        URL = 'https://www.youtube.com' + v['href']
        listVideos.append(URL)
    return listVideos

# Fonction qui nous crée notre fichier Json remplie

In [859]:
def GetCodeHTML(listeURL,chiffre):
    url = listeURL[chiffre]
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    return BeautifulSoup(webpage, 'html.parser')
    
def CréeJs(commentaire,nb_com, soup,date_commentaire) :
    nom_col = ['Airline_Name','Airline_Type','Region_Operation','Aircraft_Type','Cabin_Class','Type_Of_Lounge',
               'Type_Of_Traveller','Date_Visit','Date_Flown','Airport','Route','Category','Category_Detail',
               'Cabin_Staff_Service','Lounge_Staff_Service','Bar_And_Beverages','Food_And_Beverages','Ground_Service','Catering','Cleanliness',
              'Lounge_Comfort','Aisle_Space','Wifi_And_Connectivity','Inflight_Entertainment','Viewing_Tv_Screen','Power_Supply',
              'Seat','Seat_type','Seat_Comfort','Seat_Legroom','Seat_Storage','Seat_Width','Seat_Recline','Washrooms',
               'Value_For_Money','Overall_Customer_Rating','Overall_Service_Rating','Overall_Airline_Rating',
              'Recommended','Departure_city','Arrival_city','Nb_bus_taken','Nb_train_taken',
               'Nb_car_taken','Nb_plane_taken','Duration','Price_min','Price_max','Nb_sharing','Awards','Registration','Language']
    
    soup = soup 
    video_details = {}
    
#Fill data

    video_details['Data_Source'] = 'Youtube'
    
    for i in range (39) : 
        video_details[nom_col[i]] = ' '
    
    video_details['Date_Review'] = fctdate1(date_commentaire)
    video_details['Review'] = commentaire
    
    for i in range (39,48) :
        video_details[nom_col[i]] = ' '
        
#sortir le titre de la vidéo
    video_details['Title'] = soup.find('span',attrs={'class': 'watch-title'}).text.strip()

#sortir le nom de la chaine
    for script in soup.findAll('script',attrs={'type': 'application/ld+json'}):
        channelDescription = json.loads(script.text.strip())
        video_details['Author'] = channelDescription['itemListElement'][0]['item']['name']
        
#sortir description
    video_details['Description'] = soup.find('p',attrs={'id': "eow-description"}).text.strip()
    
#sortir la date de publication 
    video_details['Date_publication'] = str(datetime.strptime(soup.find('strong',attrs={'class': "watch-time-text"}).text.strip().replace('Ajoutée le','').replace('.',''), '%d %b %Y')). replace('00:00:00','')
#sortir le nombre de vue
    video_details['View_Count'] = (soup.find('div',attrs={'class': 'watch-view-count'}).text.strip()).replace('vues','')

#sortir un bouton likes
    for span in soup.findAll('',attrs={'class':"yt-uix-button yt-uix-button-size-default yt-uix-button-opacity yt-uix-button-has-icon no-icon-markup like-button-renderer-like-button like-button-renderer-like-button-unclicked yt-uix-clickcard-target yt-uix-tooltip"}):
        video_details['Likes']= span.find('span',attrs={'class':'yt-uix-button-content'}).text.strip()


#sortir un bouton de dislikes 
    for button in soup.findAll('button',attrs={'class':"yt-uix-button yt-uix-button-size-default yt-uix-button-opacity yt-uix-button-has-icon no-icon-markup like-button-renderer-dislike-button like-button-renderer-dislike-button-unclicked yt-uix-clickcard-target yt-uix-tooltip"}):
        video_details['Dislikes']=button.find('span',attrs={'class':'yt-uix-button-content'}).text.strip()

#sortir le nombre d'abonné
    if (soup.find('span',attrs={'class': 'yt-subscription-button-subscriber-count-branded-horizontal yt-subscriber-count'}) == None):
        video_details["Nb_subscribers"] = 0
    else :
        video_details["Nb_subscribers"] = Simplification(soup.find('span',attrs={'class': 'yt-subscription-button-subscriber-count-branded-horizontal yt-subscriber-count'}).text.strip())
        

    video_details['Nb_comments'] = (nb_com).replace('commentaires','')
    
    video_details[nom_col[48]] = ' '
#sortir hashtags
    hashtags = []

    for span in soup.findAll('span',attrs={'class': 'standalone-collection-badge-renderer-text'}):
        for a in span.findAll('a',attrs={'class': 'yt-uix-sessionlink'}):
            hashtags.append(a.text.strip())
    video_details['hashtags'] = hashtags
    
    
    for i in range (49,51):
        video_details[nom_col[i]] = ' '
    
    #detect(str(commentaire[0]))
    video_details['Language'] = 'unknown'
    if re.search("([a-z]).*", str(commentaire).lower()) :
        video_details['Language'] = detect(str(commentaire))
    

    with open('data.json', 'a', encoding='utf8') as outfile:
        json.dump(video_details, outfile, ensure_ascii=False,indent=4)

        
        
def scroll (url,nb_scroll):
    options = Options() 
    options.add_argument("--start-maximized")
    options.add_argument('headless')
    driver = webdriver.Chrome("/home/sid2019-13/Téléchargements/chromedriver",options=options)
    driver.get(url)

    Y=0
    for _ in range(nb_scroll):
        time.sleep(4)
        driver.execute_script("window.scrollTo("+str(Y)+","+str(Y+800)+")")
        Y+=800
    return BeautifulSoup(driver.page_source,'html.parser')

# Implémentation globale

In [860]:
%%time
# Create a new json

with open('data.json', 'w', encoding='utf8') as outfile: json

for Equation in equations[:1] :
    listVideos = listeUR (Equation)
    #for URLunique in range(len(listVideos)-27):
    for URLunique in range(1):
        soup1 = scroll(listVideos[URLunique],4)
        SoupCréeJS = GetCodeHTML(listVideos,URLunique)

    #date comment

        date1 = []
        for span1 in soup1.findAll('yt-formatted-string',attrs={'class':"published-time-text above-comment style-scope ytd-comment-renderer"}):
            a = (span1.find('a',attrs={'class': 'yt-simple-endpoint style-scope yt-formatted-string'}).text.strip())
            date1.append(a)  

        suivi_date = 0
        for span in soup1.findAll('yt-formatted-string',attrs={'class': 'style-scope ytd-comment-renderer'}):
            if span.text.strip() != '':
                commentaire = span.text.strip()
                nb_com = soup1.find('yt-formatted-string',attrs={'class': 'count-text style-scope ytd-comments-header-renderer'}).text.strip()
                CréeJs(commentaire,nb_com,SoupCréeJS,date1[suivi_date])
                suivi_date += 1

        print('vidéo numéro : ',URLunique, ' fini')
    print('équation fini')
print('extraction complete')

vidéo numéro :  0  fini
équation fini
extraction complete
CPU times: user 3.97 s, sys: 229 ms, total: 4.2 s
Wall time: 23.8 s
